# INF 8215 - Intelligence artif.: méthodes et algorithmes 
## Fall 2018 - TP1 - Research Methods 
### Team Components
    - David-Alexandre Beaupré - 1738405
    - Rémi Croteau - 1773718
    - Gabriel Descôteaux - 1736469



## Biking in Montreal
Approximately 10 million tourists visit Montreal every year. Concerned in providing the best experience during their stay, the tourism department of the city has initialized the development of a new application to help the visitors meet the main touristic attractions of the city in the shortest amount of time. They discovered that, when visiting the city’s attractions, the preferred mean of transportation adopted by the tourists is the bicycle. However, since the tourists do not know the city very well, they usually struggle to create their route in order to visit the planned sights for the day. Thus, given a list of places that the tourist wants to visit, including a starting and ending location, the new application determines the itinerary that passes only once in all attractions defined by the user such that the time spent within his/her route is minimum. 

<img src="images/montreal.png" alt="" width="800"/>

Our job in this TP is to develop this new application. To this end, we will address the problem using three different research methods:
1. The first methodology works by defining and exploring a search tree in order to find the route with minimum time
2. Next, an A* algorithm will be designed to search the optimal route in this search tree more efficiently 
3. Last, a heuristic procedure will be created using the Variable Neighborhood Search (VNS) metaheuristic to optimize the problem

## Problem representation

We can represent our problem using a directed and complete graph, $G(V, A)$, where each city attraction is represented by a vertex in the set of vertices $V$, and $A$ is the set of arcs representing the bicycle path between two attractions. Each arc $a_{ij} \in A$ has a cost $ w(a_{ij}) $ associated with the time spent on the path from attraction $i$ to attraction $j$. However, it is worth mentioning that the times are not symmetric, hence, is **not** guaranteed that $w(a_{ij}) = w(a_{ji})$. 

The user of the tool informs the list of $m$ attractions that he/she wishes to visit, $P = \{p_1,...,p_m\} $, where $p_1$ and $p_m$ are the starting and ending attractions, respectively.

## 1. Defining and exploring a search tree (5 points)

Let us define a search tree, $\mathcal{T}$, where each node of this tree represents a solution $S$ that is being built. Moreover, let $V(S) \subseteq V$ be the set of vertices already **visited** and $A(S) \subset A$ the set of arcs already **selected**. Thus, the cost of $S$ is given by

$$g(S) = \sum_{a \in A(S)} w(a)$$

The root node of the search tree $\mathcal{T}$ is a solution $S_{root}$ containing only the starting vertex marked as visited, i.e., $V(S_{root})=\{p_1\}$ and $A(S_{root}) = \emptyset$.

<img src="images/tree1.png" alt="" width="100"/>

Then, children nodes are created by adding an arc from $p_1$ to every vertex $ v \in P\backslash V(S_{root})$. However, an arc to the ending attraction can only be added if $p_m$ is the only vertex not visited yet. We denote the new visited vertex as $c$, the current vertex of the solution, i.e., $V(S) \gets V(S) \cup \{c\}$. 

<img src="images/tree2.png" alt="" width="400"/>

Next, each child node similarly generates its children. For a given child solution $S$ with last added vertex $c$, we add an arc $(c, v)$ to every vertex $v \in P\backslash V(S)$. The constraint to add vertex $p_m$ to the solution is kept during the entire process.

<img src="images/tree3.png" alt="" width="600"/>

The leaf nodes are complete solutions, i.e., $V(S) = P$, with $p_m$ being the last attraction added to the itinerary.  

### 1.1 Time to code

The code cell below is provided to you with a function to read the graph representing the attractions of Montreal. The function $\mathtt{read\_graph}$ returns a numpy array object $|V|\times |V|$ where the entry $\mathtt{graph[i,j]}$ contains the time spent biking from attraction $i$ to attraction $j$.



In [1]:
import numpy as np

def read_graph():
    return np.loadtxt("montreal", dtype='i', delimiter=',')

graph = read_graph()

Our first implementation task is to code the class that represents a solution. The code of class Solution below has a constructor (already implemented) that receives the list of $\mathtt{places}$ representing the set of attractions $P$ and the graph $G$, and creates a blank solution by initializing the following variables:

- $\mathtt{g}$: stores the cost of the solution
- $\mathtt{c}$: indicate the index of the current (last added) vertex of the solution
- $\mathtt{V}$: a list representing $V(S)$
- $\mathtt{unvisited}$: a list representing $P\backslash V(S)$

Alongside the constructor, there is the method $\mathtt{add}$ which is responsible for adding an arc into the solution. This method receives the index from the $\mathtt{unvisited}$ list indicating the attraction that will be added. This method also updates the $\mathtt{V}$ and $\mathtt{unvisited}$ lists, as well the cost of the solution and vertex $c$.

Now, implement the method $\mathtt{add}$ below.

In [2]:
import copy

MAX = np.iinfo(np.int32).max

# other classes useful for Edmond's algorithm implementation
class Node:
    def __init__(self, id):
        self.id = id


class Edge:
    def __init__(self, id, start, end, cost):
        self.id = id
        self.start = start
        self.end = end
        self.cost = cost

    def update(self, id, start, end, cost):
        self.id = id
        self.start = start
        self.end = end
        self.cost = cost


class Tree:
    def __init__(self, nodes=set(), edges=set()):
        self.nodes = nodes
        self.edges = edges

    def add_node(self, node):
        for vertex in self.nodes:
            if vertex.id == node.id:
                raise Exception('Node already there')
        self.nodes.add(node)

    def add_edge(self, edge):
        if edge.start in self.nodes and edge.end in self.nodes:
            for arc in self.edges:
                if arc.start == edge.start and arc.end == edge.end or arc.id == edge.id:
                    raise Exception('Edge already exists')
            self.edges.add(edge)
        else:
            raise Exception("Edge not present in nodes")


class Solution:
    def __init__(self, places, graph):
        """
        places: a list containing the indices of attractions to visit
        p1 = places[0]
        pm = places[-1]
        """
        self.g = 0  # current cost
        self.h = 0  # heuristic
        self.graph = graph
        self.visited = [places[0]]  # list of already visited attractions
        self.not_visited = copy.deepcopy(places[1:])  # list of attractions not yet visited

    def __lt__(self, other):
        return self.g + self.h < other.g + other.h

    def add(self, idx):
        """
        Adds the point in position idx of not_visited list to the solution
        """
        node = self.not_visited[idx]
        last_node = self.visited[-1]

        self.visited.append(node)
        self.not_visited.remove(node)
        self.g += self.graph[last_node, node]

    def swap(self, id1, id2):
        """
        Swaps indices and updates solution
        """
        temp_node = self.visited[id1]
        self.visited[id1] = self.visited[id2]
        self.visited[id2] = temp_node

        self.g = 0
        for idx in range(0, len(self.visited) -1):
            self.g += self.graph[self.visited[idx], self.visited[idx + 1]]

Our next step is to implement a research method to find the best solution in the search tree. A simple strategy to this end would be a [Breadth-first search](https://moodle.polymtl.ca/pluginfile.php/444662/mod_resource/content/1/recherche_en_largeur.mp4) (BFS), which explores the nodes in the sequence as they are scanned using a queue data structure. 

Now, implement the $\mathtt{bfs}$ method below that receives the graph and the list of attractions and returns the solution with minimum cost.  

In [3]:
from queue import Queue

def bfs(graph, places):
    """
    Returns the best solution which spans over all attractions indicated in 'places'
    """
    solution = Solution(places, graph)
    return_sol = copy.deepcopy(solution)
    return_sol.g = np.iinfo(np.int32).max
    solutions = Queue()
    solutions.put(solution)
    while not solutions.empty():
        current_solution = solutions.get()

        if len(current_solution.visited) == len(places) - 1:
            current_solution.add(0)
            solutions.put(current_solution)
        elif len(current_solution.visited) == len(places) and current_solution.g < return_sol.g:
            return_sol = current_solution
        else:
            for idx in range(0, len(current_solution.not_visited) - 1):
                new_sol = copy.deepcopy(current_solution)
                new_sol.add(idx)
                solutions.put(new_sol)

    return return_sol
    

### 1.2 Experiments

We propose three illustrative examples to test our search in width. The first example takes into account 7 attractions, the second 10 and the last 11. Since this research lists all possible solutions, the third example may take a considerable time to complete.

Implement these experiments and report the number of nodes explored as well as the required computing time.

In [4]:
import time 

#test 1  --------------  OPT. SOL. = 27
start_time = time.time()
places=[0, 5, 13, 16, 6, 9, 4]
sol = bfs(graph=graph, places=places)
print(sol.g)
print("--- %s seconds ---" % (time.time() - start_time))

27
--- 0.06397795677185059 seconds ---


In [5]:
#test 2 -------------- OPT. SOL. = 30
start_time = time.time()
places=[0, 1, 4, 9, 20, 18, 16, 5, 13, 19]
sol = bfs(graph=graph, places=places)
print(sol.g)
print("--- %s seconds ---" % (time.time() - start_time))

30
--- 4.964068651199341 seconds ---


In [6]:
#test 3 -------------- OPT. SOL. = 26
start_time = time.time()
places=[0, 2, 7, 13, 11, 16, 15, 7, 9, 8, 4]
sol = bfs(graph=graph, places=places)
print(sol.g)
print("--- %s seconds ---" % (time.time() - start_time))

26
--- 41.49147152900696 seconds ---


## 2. Solving with an A* algorithm (7.5 points)

For our second research method, instead of enumerating all solutions, we intend to guide the search by evaluating the potential of each node of the search tree, prioritizing the solutions with lower estimate costs. To this purpose, each solution $S$ is evaluated by the heuristic function $f(S) = g(S) + h(S) $ that takes into consideration both the current cost of $S$, $g(S)$, and an estimate of the cost (lower bound), $h(S)$, to visit the ending attraction $p_m$ from $S$.

For a given solution $S$, with $ c $ being the last added vertex to $V(S)$, the function $h$ could be defined as:

$$h(S) = \text{the shortest path from } c \text{ to } p_m \text{ in the graph } G_S  \text{ induced by the set of vertices } P\backslash V(S) \cup \{c\} $$

In other words, $h(s)$ is a lower bound to the time required to the tourist to arrive at the last attraction, $p_m$, from the current attraction, $c$, passing only through attractions of $P$ still not visited.


Hence, we can design our A* algorithm dividing it into the following steps:

1. As prevously done, the search tree $\mathcal{T}$ is initialized with a root solution $S_{root}$ that contains only the starting point, i.e., $V(S_{root}) = p_1, A(S_{root}) = \emptyset $ and $ c = p_1 $.  The cost of this initial solution is $ g(S_{root}) = 0 $. The estimative for this root solution is not important and can be discarded ($h(S_{root}) = 0 $).

2. Select the solution $S_b$ of  $\mathcal{T}$ that minimizes the heuristic function $f$:
$$ f(S_b) \leq f(S) \quad \forall S \in \mathcal{T} \qquad S_B, S \text{ are nodes of } \mathcal{T} \text{ not yet selected}$$
Note that, at the first iteration, only the $S_{root}$ is present in $\mathcal{T}$ and, thus, it will be selected. If $S_b$ is complete, then the optimal solution of the problem is found, and the algorithm ends. Otherwise, the algorithm proceeds normally to step 3.

3. Let $c$ be the last visited vertex in $S_b$. For each vertex $ v \in P\backslash V(S_b) $, perform the following tasks:
  - Create a new solution $S_n$ by adding the arc $ (c, v) $ to the solution
  - Update the sets $V(S_n)$ and $A(S_n)$;
  - Compute $g(S_n)$ and $h(S_n)$;
  - Insert $S_n$ as a new child node in $\mathcal{T}$
  
  The vertex $v = p_m$ can be only added to a new solution if it is the only vertex remaining in the set $ P\backslash V(S_b)$. 

4. Repeat steps 2 and 3 until the selected solution $S_b$ is **complete**, i.e., $V(S_B) = P$.

### 2.1 Time to code

Our first implementation task is to increment the code of the class Solution. As we shall see later, a priority queue (heap) is going to be used to implement the search tree T, instead of an ordinary queue previously used. Then, we must prepare the class Solution with a method able to compare two solutions. This is done with the method $\_\_\mathtt{lt}\_\_\mathtt{(self, other)}$ which must return true if $f(self) < f(other)$.

Now, implement the function $\_\_\mathtt{lt}\_\_\mathtt{(self, other)}$ in the class Solution.

Following that, the next step is to implement the estimation function. To compute the fastest path between two vertices of a graph we can use the [Dijkstra algorithm](https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm) where the source node is the last added vertex, $c$. The Dijkstra algorithm can be adapted to finish as soon as the fastest path to vertex $p_m$ is found.

**Implementation guidelines**:
  - Apply the Dijkstra algorithm to find the fastest path from $c$ to $p_m$.
  - Return the time spent in the path found.

In [7]:
import heapq

def fastest_path_estimation(sol):
    """
    Returns the time spent on the fastest path between
    the current vertex c and the ending vertex pm
    """
    c = sol.visited[-1]
    if not sol.not_visited:
        return 0
    pm = sol.not_visited[-1]
    nodes_to_visit = set(sol.not_visited)
    distances = {}
    for node in nodes_to_visit:
        distances[node] = np.iinfo(np.int32).max
    nodes_to_visit.add(c)
    distances[c] = 0
    while len(nodes_to_visit) > 0:
        min_node = min(nodes_to_visit, key=lambda k: distances[k])
        if min_node == pm:
            return distances[pm]
        nodes_to_visit.remove(min_node)
        for node in nodes_to_visit:
            dist = distances[min_node] + sol.graph[min_node, node]
            if dist < distances[node]:
                distances[node] = dist
                
    raise Exception("Dijkstra has failed us :( XD ...")

Finally, we must implement the A* function that creates the solution tree and performs the search. To implement the search, we will use a priority queue data structure (heap) to automatically select the solution $S_b$ from $\mathcal{T}$.

**Implementation guidelines**:
  - While a complete solution is not selected from the search tree $\mathcal{T}$ do:
      - Selected the top solution of the heap.
      - Perform step 3 described above
  - Return the optimal solution obtained.

In [8]:
def A_star(graph, places, dijkstra):
    """
    Performs the A* algorithm
    """
    # blank solution
    root = Solution(graph=graph, places=places)

    # search tree T
    T = []
    heapq.heapify(T)
    heapq.heappush(T, root)
    while len(T) > 0:
        current_solution = heapq.heappop(T)

        if len(current_solution.visited) == len(places) - 1:
            current_solution.add(0)
            heapq.heappush(T, current_solution)
        elif len(current_solution.visited) == len(places) :
            return current_solution
        else:
            for idx in range(0, len(current_solution.not_visited) - 1):
                new_sol = copy.deepcopy(current_solution)
                new_sol.add(idx)
                # Dijsktra
                if dijkstra:
                    new_sol.h = fastest_path_estimation(new_sol)
                # Edmonds
                else:
                    new_sol.h = minimum_spanning_arborescence(new_sol)
                heapq.heappush(T, new_sol)

    raise Exception("A_star has failed us :( XD ...")

### 2.2 Experiments

To test our A* algorithm, a fourth experiment with 15 attractions was included in the set of experiments. Once again, for each test, report the time of execution and the number of nodes explored in the search. 

In [9]:
#test 1  --------------  OPT. SOL. = 27
start_time = time.time()
places=[0, 5, 13, 16, 6, 9, 4]
astar_sol = A_star(graph=graph, places=places, dijkstra=True)
print(astar_sol.g)
print(astar_sol.visited)
print("--- %s seconds ---" % (time.time() - start_time))

27
[0, 5, 13, 16, 6, 9, 4]
--- 0.018535375595092773 seconds ---


In [10]:
#test 2  --------------  OPT. SOL. = 30
start_time = time.time()
places=[0, 1, 4, 9, 20, 18, 16, 5, 13, 19]
astar_sol = A_star(graph=graph, places=places, dijkstra=True)
print(astar_sol.g)
print(astar_sol.visited)
print("--- %s seconds ---" % (time.time() - start_time))

30
[0, 1, 4, 5, 9, 13, 16, 18, 20, 19]
--- 0.19760894775390625 seconds ---


In [11]:
#test 3  --------------  OPT. SOL. = 26
start_time = time.time()
places=[0, 2, 7, 13, 11, 16, 15, 7, 9, 8, 4]
astar_sol = A_star(graph=graph, places=places, dijkstra=True)
print(astar_sol.g)
print(astar_sol.visited)
print("--- %s seconds ---" % (time.time() - start_time))

26
[0, 2, 7, 7, 9, 13, 15, 16, 11, 8, 4]
--- 0.3690047264099121 seconds ---


In [12]:
#test 4  --------------  OPT. SOL. = 40
start_time = time.time()
places=[0, 2, 20, 3, 18, 12, 13, 5, 11, 16, 15, 4, 9, 14, 1]
astar_sol = A_star(graph=graph, places=places, dijkstra=True)
print(astar_sol.g)
print(astar_sol.visited)
print("--- %s seconds ---" % (time.time() - start_time))

40
[0, 3, 5, 13, 15, 18, 20, 16, 11, 12, 14, 9, 4, 2, 1]
--- 682.5311272144318 seconds ---


### 2.3 A tighter lower bound

While the estimated cost of the fastest path helped the $A*$ algorithm to solve problems with a larger number of attractions, this lower bound does not takes into consideration the cost of visiting the other vertices in $P\backslash V(S)$. A better estimation for our problem can be given by the **spanning arborescence of minimum weight** (this is the minimum spanning tree version for directed graphs) for the graph $G_S$ induced by the set of vertices $P\backslash V(S) \cup \{c\} $. This problem seeks to create a tree rooted in a defined vertex, $r$, that visits all vertices of the graph with minimum cost.

For our problem, one could define the root vertex as the current vertex of the solution, $r = c$, and compute the sum of costs required to reach the vertices $P\backslash V(S)$. A classical procedure to solve this problem is the [Edmonds's algorithm](https://en.wikipedia.org/wiki/Edmonds%27_algorithm). Our task now is to implement this algorithm and use it as a new estimation cost, $h(S)$. After that, re-run the experiments in Section 2.2 and report the gains, concerning both the execution times and the number of explored nodes.




In [13]:
def minimum_spanning_arborescence(sol):
    root = sol.visited[-1]
    nodes = set()
    edges = set()
    idx = 0
    vertexes = set()
    for vertex in sol.not_visited:
        found = False
        for vert in vertexes:
            if vert.id == vertex:
                found = True
        if not found:
            vertexes.add(Node(vertex))

    root_in_vertices = False
    for vert in vertexes:
        if vert.id == root:
            root_in_vertices = True
    if not root_in_vertices:
        vertexes.add(Node(root))
    for start_vertex in vertexes:
        for end_vertex in vertexes:
            if start_vertex != end_vertex and end_vertex.id != root:
                edge = Edge(idx, start_vertex, end_vertex, sol.graph[start_vertex.id, end_vertex.id])
                edges.add(edge)
                idx += 1
        nodes.add(start_vertex)
    tree = Tree(nodes, edges)
    id_vertex = max(vertexes, key=lambda x: x.id).id + 1
    id_edge = idx

    min_span_tree = recursive_edmonds(tree, root, id_vertex, id_edge)
    
    cost = 0
    for edge in min_span_tree.edges:
        cost += edge.cost
        
    return cost


def recursive_edmonds(tree, root, id_vertex, id_edge):
    best_in_edges = {}
    kicks_out = {}
    real = {}
    for vertex in tree.nodes:
        if vertex.id != root:
            min_edge = None
            min_cost = MAX
            for edge in tree.edges:
                if edge.end == vertex and edge.cost < min_cost:
                    min_cost = edge.cost
                    min_edge = edge
            best_in_edges[vertex] = min_edge
            
            # look for cycle
            start_node = vertex
            cycle = [start_node]
            cycle_edges = []
            cycle_found = False
            while start_node is not None and not cycle_found:
                from_node = best_in_edges[start_node].start
                if from_node in best_in_edges:
                    cycle_edges.append(best_in_edges[start_node])
                    start_node = from_node
                    cycle.append(start_node)
                else:
                    start_node = None
                if start_node == vertex:
                    cycle_found = True
                    cycle.pop()
                   
            if cycle_found:
                new_vertex = Node(id_vertex)
                id_vertex += 1
                subtree = Tree(set(), set())
                for node in tree.nodes:
                    if node not in cycle:
                        subtree.add_node(node)
                subtree.add_node(new_vertex)
                sub_edges = set()
                for edge in tree.edges:
                    new_edge = Edge(0, 0, 0, 0)
                    if edge.start not in cycle and edge.end not in cycle:
                        new_edge.update(id_edge, edge.start, edge.end, edge.cost)
                        id_edge += 1
                        real[new_edge] = edge
                        sub_edges.add(new_edge)
                    elif edge.start in cycle and edge.end not in cycle:
                        new_edge.update(id_edge, new_vertex, edge.end, edge.cost)
                        id_edge += 1
                        real[new_edge] = edge
                        sub_edges.add(new_edge)
                    elif edge.start not in cycle and edge.end in cycle:
                        new_edge.update(id_edge, edge.start, new_vertex, edge.cost)
                        kicks_out[new_edge] = best_in_edges[edge.end]
                        new_edge.cost -= best_in_edges[edge.end].cost
                        id_edge += 1
                        real[new_edge] = edge
                        sub_edges.add(new_edge)

                subtree.edges = sub_edges
                arborescence = recursive_edmonds(subtree, root, id_vertex, id_edge)
                
                return_tree = Tree(set(), set())
                
                kicked_out = None
                for edge in arborescence.edges:
                    if edge in real:
                        real_edge = real[edge]
                        if real_edge.start not in return_tree.nodes:
                            return_tree.add_node(real_edge.start)
                        if real_edge.end not in return_tree.nodes:
                            return_tree.add_node(real_edge.end)
                        if real_edge not in return_tree.edges:
                            return_tree.add_edge(real_edge)
                    if edge.end == new_vertex:
                        kicked_out = kicks_out[edge]
                for edge in tree.edges:
                    if edge in cycle_edges and edge != kicked_out:
                        if edge.start not in return_tree.nodes:
                            return_tree.add_node(edge.start)
                        if edge.end not in return_tree.nodes:
                            return_tree.add_node(edge.end)
                        return_tree.add_edge(edge)
                return return_tree
        
    cycle_free_tree = Tree(set(), set())
    for node, in_edge in best_in_edges.items():
        if node not in cycle_free_tree.nodes:
            cycle_free_tree.add_node(node)
        if in_edge.start not in cycle_free_tree.nodes:
            cycle_free_tree.add_node(in_edge.start)
        cycle_free_tree.add_edge(in_edge)
    return cycle_free_tree

In [18]:
#test 1  --------------  OPT. SOL. = 27
start_time = time.time()
places=[0, 5, 13, 16, 6, 9, 4]
astar_sol = A_star(graph=graph, places=places, dijkstra=False)
print(astar_sol.g)
print(astar_sol.visited)
print("--- %s seconds ---" % (time.time() - start_time))

27
[0, 5, 13, 16, 6, 9, 4]
--- 0.038455963134765625 seconds ---


In [19]:
#test 2  --------------  OPT. SOL. = 30
start_time = time.time()
places=[0, 1, 4, 9, 20, 18, 16, 5, 13, 19]
astar_sol = A_star(graph=graph, places=places, dijkstra=False)
print(astar_sol.g)
print(astar_sol.visited)
print("--- %s seconds ---" % (time.time() - start_time))

30
[0, 1, 4, 5, 9, 13, 16, 18, 20, 19]
--- 0.051953792572021484 seconds ---


In [20]:
#test 3  --------------  OPT. SOL. = 26
start_time = time.time()
places=[0, 2, 7, 13, 11, 16, 15, 7, 9, 8, 4]
astar_sol = A_star(graph=graph, places=places, dijkstra=False)
print(astar_sol.g)
print(astar_sol.visited)
print("--- %s seconds ---" % (time.time() - start_time))

26
[0, 2, 7, 7, 9, 13, 15, 16, 11, 8, 4]
--- 0.064422607421875 seconds ---


In [21]:
#test 4  --------------  OPT. SOL. = 40
start_time = time.time()
places=[0, 2, 20, 3, 18, 12, 13, 5, 11, 16, 15, 4, 9, 14, 1]
astar_sol = A_star(graph=graph, places=places, dijkstra=False)
print(astar_sol.g)
print(astar_sol.visited)
print("--- %s seconds ---" % (time.time() - start_time))

40
[0, 3, 13, 15, 18, 20, 16, 11, 12, 14, 9, 5, 4, 2, 1]
--- 21.130451679229736 seconds ---


In [1]:
'''
Comparaison Dijkstra - Edmond
Test 1: 0.0185s - 0.0385s
Test 2: 0.1976s - 0.0519s
Test 3: 0.3690s - 0.0644s
Test 4: 682.53s - 21.23s

À l'exception du test 1, Edmond a des temps inférieurs pour A* que Dijkstra
'''

"\nComparaison Dijkstra - Edmond\nTest 1: 0.0185s - 0.0385s\nTest 2: 0.1976s - 0.0519s\nTest 3: 0.3690s - 0.0644s\nTest 4: 682.53s - 21.23s\n\nÀ l'exception du test 1, Edmond a des temps inférieurs pour A* que Dijkstra\n"

## 2. Optimizing with a VNS heuristic (7.5 points)

This time, instead of building a solution from scratch by gradually adding arcs to the solution,  our [Variable Neighborhood Search](https://en.wikipedia.org/wiki/Variable_neighborhood_search) (VNS) method will start from a complete initial solution and performs successively swaps on the order in which the attractions are visited to improve the cost of the solution. 

<img src="images/vns.png" alt="" width="800"/>

### 2.1 Time to code

Our first implementation task is to construct an initial solution to work with during the search. This solution must be complete, i.e., we must create an ordered set of visited vertex $V(S)$ such that it starts in $p_1$ and finishes in $p_m$. A simple approach to this end is to use a [Depth-First Seach (DFS)](https://moodle.polymtl.ca/pluginfile.php/445484/mod_resource/content/1/recherche_en_profondeur.mp4). The starting point is the vertex representing $p_1$ and, as soon as the ending point is reached after visiting all the others attractions of $P$, the path can be turned into an initial solution. To help in diversifying the search, the method to generate an initial solution can be randomized such that the VNS algorithm can start the search in different regions of the solution space. Thus, in the DFS function, the selection of the child to continue the search must be random.

Now. implement the **initial_sol** method to construct a random initial solution and uses the **dfs** function to find a path from $p_1$ to $p_m$.



**Implementation guidelines**:
  - Call the dfs function to find a complete path from  $p_1$ to $p_m$.
  - Create a root solution
  - Adjust the solution cost and the set V with the path found
  - Return the solution created



In [28]:
import random
from random import shuffle, randint

def initial_sol(graph, places):
    """
    Return a completed initial solution
    """
    return dfs(graph, places)

def dfs(graph, places):
    """
    Returns the best solution which spans over all attractions indicated in 'places'
    """
    solution = Solution(places, graph)
    return_sol = copy.deepcopy(solution)
    return_sol.g = np.iinfo(np.int32).max
    solutions = list()
    solutions.append(solution)
    while len(solutions) > 0:
        current_solution = solutions.pop()

        if len(current_solution.visited) == len(places) - 1:
            current_solution.add(0)
            solutions.append(current_solution)
        elif len(current_solution.visited) == len(places):
            return current_solution
        else:
            temp_list = list()
            for idx in range(0, len(current_solution.not_visited) - 1):
                temp_list.append(idx)
            random.shuffle(temp_list)
            for idx in temp_list:
                new_sol = copy.deepcopy(current_solution)
                new_sol.add(idx)
                solutions.append(new_sol)

    raise Exception('Cuz I''m in too deep - Sum41')

A key feature of the VNS metaheuristic is the ability to define $k_{max}$ neighborhoods to diversify the search and escape from local optima. For our problem, a simple and efficient neighborhood structure could be neighborhoods based on **swapping** the order in which two points are visited within the route. Thus, given a solution $S$, with $V(S)$ being the ordered set in which the points were visited, a solution on the $k$-th neighborhood of $S$ can be obtained by selecting $k$ *random* pairs of vertices, different from $p_1$ and $p_m$, and swap their position in $V(S)$.

In the VNS framework, the procedure of generating a solution on the $k$-th neighborhood of $S$ is known as the **shaking** step. Thus, our task now is to implement this shaking method that receives a solution, the neighborhood index $k$ and the graph $G$ and returns a new solution on the $k$-th neighborhood of $S$. 

However, before implementing the shaking method below, we must first return to the implementation of class Solution and finish the **swap** method. This method is responsible for updating the position of the vertices being swapped in the $V(S)$ set and, consequentily, the solution cost. It receives the indices of the two selected vertices of $V(S)$ and the graph $G$.


**Implementation guidelines (shaking)**:
  - Select two indices, $i$ and $j$, such that $i \neq j$ and $i, j \in \{2...m-1\}$
  - Make a copy of the $sol$ and perfom the swap
  - Return the created solution

In [29]:
def shaking(sol, k):
    """
    Returns a solution on the k-th neighrboohood of sol
    """
    nb_swaps = 0
    solutions = []
    for nb_swaps in range (0,k):
        max_id = len(sol.visited) - 2
        id1 = random.randint(1, max_id)
        id2 = -1
        while id2 == -1 or id2 == id1:
            id2 = random.randint(1, max_id)
        new_sol = copy.deepcopy(sol)
        new_sol.swap(id1, id2)
        solutions.append(new_sol)
        sol = new_sol
    
    return solutions.pop()

Another essential step in the VNS algorithm is to invoke a local search routine to improve the solution cost. For our problem, an efficient local search procedure would be to search for a pair of vertices, $i$ and $j$, such that their swapping position in $V(S)$ leads it to a reduction in the solution cost. This local search is known as 2-opt, and we are going to implement in this TP. 

Formally, consider a solution $S$ and the ordered set of vertices $V(S)$. For a vertex $i$, let $i'$ be the immediate successor of $i$ in the sequence $V(S)$. The 2-opt algorithm work as follows: for every  pair of non-consecutive vertices $i,j$, check if by swapping the position of vertices $i'$ and $j$ results in a improvement in the solution cost. If so, perform this swap. This process is repeated until there are no more profitable exchanges. The 2-opt heuristic is illustrated in the figure below. 


<img src="images/2opt.png" alt="" width="800"/>

<img src="images/2opt2.png" alt="" width="800"/>



Now implement the **local_search_2opt** fuction below. 



**Implementation guidelines**:
  - Check every pair of indices, $i = \{2,..,m-3\}$ and $j = \{i+2, m-1\}$ for a improvement in the solution cost
  - If the exchange is profitable, perform it
  - Repeat the process until no more profitable swap can be found

In [30]:
def local_search_2opt(sol):
    """
    Apply 2-opt local search over sol
    """
    new_sol = copy.deepcopy(sol)
    profitable_swap_found = True
    while profitable_swap_found:
        profitable_swap_found = False
        for id1 in range (1, len(sol.visited) - 3):
            for id2 in range(id1 + 2, len(sol.visited) - 1):
                if id1 != id2:
                    temp_sol = copy.deepcopy(new_sol)
                    temp_sol.swap(id1, id2)
                    if temp_sol.g < new_sol.g:
                        new_sol = temp_sol
                        profitable_swap_found = True
                        
    return new_sol

Finally, our last implementation step is to code the VNS algorithm itself. The method receives an initial solution  and returns the best solution found during the search. Alongside the solution and the graph, the method also receives two other parameters: $k_{max} $, which defines the number of neighborhoods; and $t_{max}$, which sets the time limit to run the heuristic.


**Implementation guidelines**:
  - At each iteration of the VNS, generate a solution on the k-th neighrborhood of the current best solution and apply a local search on it.
  - If the new solution has a better cost than the current best solution, update the best solution with the new found solution. 
  - Repeat the process until the time limit is met

In [31]:
def vns(sol, k_max, t_max):
    """
    Performs the VNS algorithm
    """
    start_time = time.time()  
    current_best_sol = sol
    while t_max > time.time() - start_time:
        new_sol = shaking(current_best_sol, k_max)
        new_sol = local_search_2opt(new_sol)
        if new_sol.g < current_best_sol.g:
            current_best_sol = new_sol
            
    return current_best_sol

### 2.2 Experiments

Run the VNS on the following illustrative examples and report the obtained solutions:

In [32]:
# test 1  --------------  OPT. SOL. = 27
places=[0, 5, 13, 16, 6, 9, 4]
sol = initial_sol(graph=graph, places=places)
start_time = time.time()
vns_sol = vns(sol=sol, k_max=10, t_max=1)
print(vns_sol.g)
print(vns_sol.visited)
print("--- %s seconds ---" % (time.time() - start_time))

27
[0, 5, 13, 16, 6, 9, 4]
--- 1.000842809677124 seconds ---


In [33]:
#test 2  --------------  OPT. SOL. = 30
places=[0, 1, 4, 9, 20, 18, 16, 5, 13, 19]
sol = initial_sol(graph=graph, places=places)

start_time = time.time()
vns_sol = vns(sol=sol, k_max=10, t_max=1)
print(vns_sol.g)
print(vns_sol.visited)

print("--- %s seconds ---" % (time.time() - start_time))

30
[0, 1, 4, 5, 9, 13, 16, 18, 20, 19]
--- 1.0033533573150635 seconds ---


In [34]:
# test 3  --------------  OPT. SOL. = 26
places=[0, 2, 7, 13, 11, 16, 15, 7, 9, 8, 4]
sol = initial_sol(graph=graph, places=places)

start_time = time.time()
vns_sol = vns(sol=sol, k_max=10, t_max=1)
print(vns_sol.g)
print(vns_sol.visited)
print("--- %s seconds ---" % (time.time() - start_time))

26
[0, 2, 7, 7, 9, 13, 15, 16, 11, 8, 4]
--- 1.0005698204040527 seconds ---


In [35]:
# test 4  --------------  OPT. SOL. = 40
places=[0, 2, 20, 3, 18, 12, 13, 5, 11, 16, 15, 4, 9, 14, 1]
sol = initial_sol(graph=graph, places=places)

start_time = time.time()
vns_sol = vns(sol=sol, k_max=10, t_max=1)
print(vns_sol.g)
print(vns_sol.visited)
print("--- %s seconds ---" % (time.time() - start_time))

40
[0, 3, 5, 14, 12, 13, 15, 18, 20, 16, 11, 9, 4, 2, 1]
--- 1.00700044631958 seconds ---


## 4. BONUS SECTION (1 point)

Explain in which situation each of the developed algorithms is more appropriate (hint: take into consideration the scalability of the problem)

In [2]:
'''
** Fichier PDF dans l'archive remise **
--ENGLISH VERSION FOLLOWS--
Dans la plupart des cas, lorsque l'arbre de recherche est équilibré, il semble peu probable que la recherche en largeur soit supérieure en termes de performance. En fait, si toutes les feuilles se trouvent au même niveau, une recherche en largeurva toujours explorer l'arbre dans son entièreté avant de retourner une solution. Il s'agit alors d'un O(bm), où b est le nombre d'enfants que chaque noeud possède, et m est le nombre d'étages de l'arbre. Une recherche en largeurpeut toutefois être utile si l'arbre n'est pas équilibré, si aucune bone heuristique n'est trouvée (voir ci-dessous), or si la solution optimale se trouve sur l'une des branches les plus courtes de l'arbre. Par exemple, si on souhaite se rendre de A vers B sur une grille infinie où toutes les intersections de la grille sont des noeuds et la distance entre chaque noeud est constante, il est évident que le chemin le plus court serait celui passant par le plus petit nombre de noeuds; c'est d'ailleurs la solution qu'on obtiendrait avec le recherche en largeur. Toutefois, comme la recherche en largeur explorer la même distance dans toutes les directions, on pourrait alors trouver un moyen plus efficace de trouver une solution en obtenant une idée générale de la direction à prioriser.
L'algorithme A* utilise une heuristique qui permet ainsi de s'orienter dans la bonne direction, offrant ainsi une performance supérieure à la recherche en largeur lorsque l'heuristique est bonne. Une bonne heuristique en est une qui peut être calculée relativement rapidement et qui permet de pointer dans une bonne direction avec une forte probabilité; une mauvaise heuristique, par exemple, en serait une qui explore elle-même l'arbre de recherche dans son entièreté. La première heuristique implémentée dans le cadre du laboratoire est celle de Dijkstra. Elle a permis d'obtenir une solution beaucoup plus rapidement que celle implémentée avec la recherche ne largeur. Selon nous, ce résultat peut s'étendre à la majorité des cas semblables à celui étudié pour les raisons mentionnées ci-dessus. La deuxième heuristique implémentée, soit l'algorithme d'Edmond, a offert des résultats encore supérieurs à ceux avec Dijkstra. Si les deux heuristiques étaient des aussi bons indicateurs de la direction à prioriser, ce qui n'est pas la cas ici avec Edmond étant légèrement supérieur, alors l'élément différentiatif entre les deux serait la complexité de l'algorithme et le temps d'exécution. Dans cette situation, Dijkstra serait à favoriser, offrant un temps d'exécution de O(E + VlogV), contrairement à O(EV).
Enfin, la recherche par voisinage variable (VNS) est une alternative intéressante. Elle a l'avantage de permetre de trouver une solution très rapidement, puis d'essayer d'améliorer cette solution. Selon nous, si on cherche simplement une bonne solution, mais pas nécessairement la meilleure solution, globalement, le VNS est la méthode qui se généralise le plus aux problèmes à plus grande échelle. En fait, pour un graphe très grand, A* ou la recherche en largeur sont excessivement exigeants au niveau de l'utilisation de la mémoire et du processeur (CPU), alors que le VNS permet de trouver une solution généralement bonne en un temps décent. Aussi, l'algorithme possède la propriété intéressante qu'il est possible de l'arrêter à tout moment afin d'obtenir la meilleure solution trouvée jusqu'à présent.

For most cases where the search tree is well balanced, it seems unlikely that the BFS algorithm might be superior in terms of performance. In fact, if all leaves are found at the same level, a breadth-first search will have to look through almost the entire tree before returning its solution. This would be O(bm), where b is the number of children each node has, and m is the number of levels the tree has. A BFS could prove useful in cases where the tree is seriously unbalanced, no good heuristics have been found, and especially if the best solutions tend to be leaves to the shortest branches. An example of this would be a case where something needs to go from A to B on an infinite grid where all the intersections on the grid are considered nodes, and the distance between each node is constant. Clearly, the best (shortest) path would be the one that requires passing through the smallest number of nodes (shortest branch of the search tree), and that is in fact the solution a BFS would return. Again however, that would only be acceptable if there were no good known heuristics to use, because until a BFS finds a solution, it will have looked at the same depth in all different directions, when there are normally simple ways to get a general idea of the direction to prioritize.
The A* star incorporates a heuristic which taps into the aforementioned idea of prioritizing a direction that is likely to lead to the solution faster. When that heuristic is good, this will generally prove far superior to a BFS for instance. Here we define a “good” heuristic as one that can be computed rather fast (an extreme counter-example would be a “heuristic” which itself searches the whole tree, ultimately leading to no gain whatsoever), and which has a very high probability of pointing in a direction that leads to a decent, if not optimal, solution. The first heuristic we implemented for our A* algorithm, the fastest path to endpoint (Dijkstra algorithm), has proved able to find a solution much faster than a BFS. We believe this finding would extend to most cases similar to the one studied for reasons explained in the paragraph on BFS. Evidently though, the fastest path to endpoint can be misleading, as it does not account for all the other nodes to visit before reaching the arrival. It is in that sense that the other heuristic we implemented for A*, the minimal spanning tree, is preferable, and does in fact prove faster in our tests. If both heuristics were about equally good indicators of the direction to prioritize, then the differentiating factor would be their own algorithmic complexity. That would normally crown Dijkstra instead, which typically runs in O(E + V log V) as opposed to O(EV).12
Finally, the variable neighborhood search (VNS) is an interesting alternative. It has the advantage of allowing for a complete solution very quickly, before going on to improve it. We are inclined to believe that, granted we do not need the globally optimal solution, it is the most scalable of the algorithms we implemented in this lab. In fact, finding the fastest path or a minimal spanning tree at every iteration of an A* run (or worse, performing a BFS) on a massive graph seems quite dreadful in terms of CPU usage. However, a metaheuristic such as VNS can find an acceptable solution in a decent amount of time. Furthermore, the VNS has the interesting property that a time limit for convergence can be set, since at any given time, it holds a complete best solution so far.
1 https://en.wikipedia.org/wiki/Edmonds%27_algorithm
2 https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm
'''

"\n** Fichier PDF dans l'archive remise **\n--ENGLISH VERSION FOLLOWS--\nDans la plupart des cas, lorsque l'arbre de recherche est équilibré, il semble peu probable que la recherche en largeur soit supérieure en termes de performance. En fait, si toutes les feuilles se trouvent au même niveau, une recherche en largeurva toujours explorer l'arbre dans son entièreté avant de retourner une solution. Il s'agit alors d'un O(bm), où b est le nombre d'enfants que chaque noeud possède, et m est le nombre d'étages de l'arbre. Une recherche en largeurpeut toutefois être utile si l'arbre n'est pas équilibré, si aucune bone heuristique n'est trouvée (voir ci-dessous), or si la solution optimale se trouve sur l'une des branches les plus courtes de l'arbre. Par exemple, si on souhaite se rendre de A vers B sur une grille infinie où toutes les intersections de la grille sont des noeuds et la distance entre chaque noeud est constante, il est évident que le chemin le plus court serait celui passant par

## 5 Due date
The TP must be submitted before Octobre 6th 23h55.  Any late work will be penalized with a value of 10% per day of delay.